SimCLR study on Cifar10 data. 

Network: ResNet18 

Batch size: 256

Optimizer: SGD

Loss: NT-Xent loss

lr:  0.3

Pre-Training data: 10000 images (1000 image per class)

Trained for 300 epochs

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torch.nn as nn
from torch.optim import lr_scheduler
import copy
import time
import os
import argparse
from torch.utils.data import Subset
import csv
device = torch.device('cuda')
from ResNet import *
import matplotlib.pyplot as plt
#tqdm
from tqdm import tqdm

In [ ]:
loaded_data = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/test_data.pt")
loaded_data1 = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/data.pt")

In [ ]:
index = loaded_data['index']
images1 = loaded_data['images']
labels1 = loaded_data['labels']
#print shape of tensors
# images1=images1
# images1 = images1 / 255
#     # mean normalize
# images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
d_images1=images1[:5000]
d_labels1=labels1[:5000]
print(d_images1.shape)
print(d_labels1.shape)


In [ ]:
image=loaded_data1['images']
label=loaded_data1['labels']
# image=image/255
# image=(image-image.mean(dim=(0,2,3),keepdim=True))/image.std(dim=(0,2,3),keepdim=True)
b_image1=image[:10000]
b_label1=label[:10000]
print(b_image1.shape)
print(b_label1.shape)

In [ ]:
# image=image[10000:]
# image1=images1[5000:]
# label=label[10000:]
# label1=labels1[5000:]

In [ ]:
pretrain_images = torch.cat((b_image1, d_images1), dim=0)
pretrain_labels = torch.cat((b_label1, d_labels1), dim=0)
pretrain_image=pretrain_images/255
pretrain_image=(pretrain_image-pretrain_image.mean(dim=(0,2,3),keepdim=True))/pretrain_image.std(dim=(0,2,3),keepdim=True)

In [ ]:
pretrain_images.shape
pretrain_labels.shape

### Augmentaion on pre-training data

In [ ]:
class Augment_train(object):
  def __init__(self,size):
    s = 0.5 # jitter strength
    # size = 224
    color_jitter = transforms.ColorJitter(0.4*s, 0.4*s, 0.4*s, 0.1*s)
    
    self.train_transforms = transforms.Compose([
        # transforms.RandomResizedCrop(size, scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([color_jitter], p=0.5), 
        transforms.RandomGrayscale(p=0.4),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
      ])
  
  def __call__(self,x):
    transform = self.train_transforms

    xi = transform(x)
    xj = transform(x)
    return xi, xj

In [ ]:
aumention = Augment_train(224)
augmented_images1,augmented_images2 = aumention(pretrain_image)

In [ ]:
augmented_images1.shape
augmented_images2.shape

In [ ]:
b_images2=image[10000:20000]
b_labels2=label[10000:20000]
b_images2.shape
b_labels2.shape

In [ ]:
d_images2=images1[5000:7000]
d_labels2=labels1[5000:7000]
d_images2.shape
d_labels2.shape

In [ ]:
pretrain_images1 = torch.cat((b_images2, d_images2), dim=0)
pretrain_labels1 = torch.cat((b_labels2, d_labels2), dim=0)
pretrain_image1=pretrain_images1/255
pretrain_image1=(pretrain_image1-pretrain_image1.mean(dim=(0,2,3),keepdim=True))/pretrain_image1.std(dim=(0,2,3),keepdim=True)

In [ ]:
augmented_images11,augmented_images22 = aumention(pretrain_image1)

In [ ]:
augmented_images11.shape

In [ ]:
b_images3=image[20000:]
b_labels3=label[20000:]
b_images3.shape
b_labels3.shape

In [ ]:
d_images3=images1[7000:]
d_labels3=labels1[7000:]
d_images3.shape
d_labels3.shape

In [ ]:
pretrain_images2 = torch.cat((b_images3, d_images3), dim=0)
pretrain_labels2 = torch.cat((b_labels3, d_labels3), dim=0)
pretrain_image2=pretrain_images2/255
pretrain_image2=(pretrain_image2-pretrain_image2.mean(dim=(0,2,3),keepdim=True))/pretrain_image2.std(dim=(0,2,3),keepdim=True)

In [ ]:
augmented_images123,augmented_images223 = aumention(pretrain_image2)

In [ ]:
augmented_images123.shape

In [ ]:
augmented_images123.shape
augmented_images1.shape
augmented_images11.shape

In [ ]:
augmented_images_T1= torch.cat((augmented_images1,augmented_images11,augmented_images123),dim=0)
augmented_images_T2= torch.cat((augmented_images2,augmented_images22,augmented_images223),dim=0)

In [ ]:
print(augmented_images_T1.shape)
print(augmented_images_T2.shape)


### Prepare the Train loader

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

print(augmented_images1.shape)
print(augmented_images2.shape)
class CustomDataset(Dataset):
    def __init__(self, images1, images2):
        self.images1 = images1
        self.images2 = images2

    def __len__(self):
        return len(self.images1)

    def __getitem__(self, idx):
        return self.images1[idx], self.images2[idx]

custom_dataset = CustomDataset(augmented_images_T1, augmented_images_T2)
print(len(custom_dataset))
trainloader = DataLoader(custom_dataset, batch_size=256 ,shuffle=True, num_workers=16)


In [ ]:
data_loader_size = len(trainloader)
print("Data loader size:", data_loader_size)


In [ ]:
import matplotlib.pyplot as plt

# Get the first batch of images from the trainloader
for i, (batch_images1, batch_images2) in enumerate(trainloader):
    if i == 0:
        break

# Plot the images
fig, axes = plt.subplots(2, 6, figsize=(10, 5))
for i in range(6):
    # Plot images from batch_images1
    image1 = (batch_images1[i] - batch_images1[i].min()) / (batch_images1[i].max() - batch_images1[i].min())
    axes[0, i].imshow(image1.permute(1, 2, 0))
    axes[0, i].axis('off')

    # Plot images from batch_images2
    image2 = (batch_images2[i] - batch_images2[i].min()) / (batch_images2[i].max() - batch_images2[i].min())
    axes[1, i].imshow(image2.permute(1, 2, 0))
    axes[1, i].axis('off')

plt.show()


### SimCLR  specific Loss function Cosine Similarity

In [ ]:
class NTXent_L(nn.Module):
  def __init__(self,batch_size,temperature=0.5):
      super().__init__()
      self.batch_size=batch_size
      self.temperature = temperature
      
  def cosine_sim(self, output1,output2): #output1, output2 as 2 augmented versions of image;
    o_con= torch.cat([output1,output2], dim=0)
    return F.cosine_similarity(o_con.unsqueeze(1), o_con.unsqueeze(0), dim=2)
  def forward(self, z_i, z_j):

    batch_size= z_i.shape[0]

    sim_m = self.cosine_sim(z_i,z_j)

    sim_ij = torch.diag(sim_m,batch_size)
    sim_ji = torch.diag(sim_m, -batch_size)
   
    sim_con= torch.cat([sim_ij, sim_ji],dim=0)
    pos= torch.exp(sim_con/self.temperature)
    mask = (~torch.eye(batch_size *2 ,batch_size*2, dtype=bool)).float()
    neg= mask.to(device) *torch.exp(sim_m/self.temperature)
    loss = torch.sum(-torch.log(pos/torch.sum(neg, dim=1))) / (2*self.batch_size)
    
    return loss

In [ ]:
criterion = NTXent_L(batch_size=256, temperature = 0.5)
criterion = criterion.to(device)

### SimCLR Model

In [ ]:
# base_encoder
base_encoder = torchvision.models.efficientnet(pretrained=True)
base_encoder 

In [ ]:
# base_encoder= ResNet50()
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x
    
    
class SimCLR_model(nn.Module):

    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()

        self.base_encoder = base_encoder
        self.n_features = n_features
        self.base_encoder.fc=nn.Linear(n_features, n_features)
        self.base_encoder.linear=Identity()
        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )
        # self.base_encoder.fc=Identity()

    def forward(self, x_i, x_j):
        h_i = self.base_encoder(x_i)
        h_j = self.base_encoder(x_j)
        z_i = self.projectionHead(h_i)
        z_j = self.projectionHead(h_j)
       
        return h_i, h_j, z_i, z_j
    
projection_dim=128
n_features= base_encoder.fc.in_features
net= SimCLR_model(base_encoder, projection_dim, n_features)
net= net.to(device)
batch_size = 512
lr = 0.3
start_epoch=0
max_epoch=300
# optimizer= torch.optim.Adam(net.parameters(), lr)
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-6)

In [ ]:
net.to(device)

### Train the Pre-text task

In [ ]:
net.base_encoder.fc.weight
# a=net.base_encoder.classifier[1].weight
# a.shape

In [ ]:
train_losses = []
def train(epoch):
    net.train()
    s = 0
    tot_loss = 0
    total = 0
    batch_idx = 0
    i = 0
    for batch_idx, (images) in enumerate(trainloader):
        images[0], images[1] = images[0].to(device), images[1].to(device)
        optimizer.zero_grad()

        h_i, h_j, z_i, z_j = net(images[0], images[1])
        loss = criterion(z_i, z_j)

        loss.backward()

        optimizer.step()

        tot_loss += loss.item()

    # print("epoch=", epoch)
    # print("epoch loss=", tot_loss)
    train_losses.append(tot_loss)

    if epoch % 10 == 0:
        modelstate_save_name = 'modelstate_bs_restart_r_bk_DB_resnet50_3.pt'
        path_modelstate = F"/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/{modelstate_save_name}"
        torch.save(net.state_dict(), path_modelstate)

        parameters_end = {
            'net': net.state_dict(),
            'epoch': epoch,
            'loss': tot_loss
        }
        paramsend_save_name = 'paramsend_bs_restart_r_bk_DB_resenet50_3.pt'
        path_paramsend = F"/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/{paramsend_save_name}"
        torch.save(parameters_end, path_paramsend)
        print("epoch=", epoch)
        print("epoch loss=", tot_loss)

net.to(device)

for epoch in tqdm(range(start_epoch, start_epoch + 301)):
    train(epoch)


In [ ]:
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss over Epochs')
plt.show()